In [1]:
import datetime
import pandas as pd

In [2]:
# Data Structure is garbage, ecah line ends with a separator... so will have to import columnames separately
# Secondly, there's a change in data structure partway through... from 28 elements per line to 37... happens on 
# Line 29 (1-indexed, line 28 if 0-indexed)

# first get Data
struct_change_line=28
df = pd.read_csv('data/workout-data/spreadsheet.stronglifts-fixed_notes.csv', nrows=struct_change_line-1, skiprows=1, header=None)
df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,02/09/15,NaN,A,175LB,NaN,42.5,95,5,5,5,...,5,NaN,42.5,95,5,5.0,5.0,5.0,5.0,NaN
1,02/11/15,NaN,B,175LB,NaN,45.0,100,5,5,5,...,5,NaN,60.0,135,5,NaN,NaN,NaN,NaN,NaN


In [3]:
# Fix the idiotic column isusue due to an extra delimiter per line...
# Fill in with column number
col_names = pd.read_csv('data/workout-data/spreadsheet.stronglifts-fixed_notes.csv', nrows=0).columns.tolist()
n_cols0 = len(col_names)
n_cols1 = df.shape[1]
col_names = col_names + list(range(n_cols0, n_cols1))

df.columns = col_names
# and parse the dates
df['Date'] = pd.to_datetime(df['Date']).apply(lambda d: d.date())
df.head(2)

,Date,Note,Workout,Body Weight,Exercise 1,Weight (KG),Weight (LB),Set 1,Set 2,Set 3,...,Set 5.1,Exercise 3,Weight (KG).2,Weight (LB).2,Set 1.2,Set 2.2,Set 3.2,Set 4.2,Set 5.2,28
0,2015-02-09,NaN,A,175LB,NaN,42.5,95,5,5,5,...,5,NaN,42.5,95,5,5.0,5.0,5.0,5.0,NaN
1,2015-02-11,NaN,B,175LB,NaN,45.0,100,5,5,5,...,5,NaN,60.0,135,5,NaN,NaN,NaN,NaN,NaN


In [4]:
# Read in the rest of the data
df1 = pd.read_csv('data/workout-data/spreadsheet.stronglifts-fixed_notes.csv', skiprows=struct_change_line, header=None)

# And fix column names in similar fashion
col_names = df.columns.tolist()
n_cols0 = len(col_names)
n_cols1 = df1.shape[1]

col_names = col_names + list(range(n_cols0, n_cols1))
df1.columns = col_names
# And parse the dates
df1['Date'] = pd.to_datetime(df1['Date']).apply(lambda d: d.date())

df1.head(2)

,Date,Note,Workout,Body Weight,Exercise 1,Weight (KG),Weight (LB),Set 1,Set 2,Set 3,...,Set 5.2,28,29,30,31,32,33,34,35,36
0,2015-04-14,NaN,B,168.8LB,NaN,97.5,215,5.0,5.0,5.0,...,NaN,NaN,0.0,0.0,8.0,5.0,3.0,NaN,NaN,NaN
1,2015-04-15,NaN,A,168.8LB,NaN,100.0,220,5.0,5.0,5.0,...,5.0,NaN,0.0,0.0,5.0,5.0,6.0,NaN,NaN,NaN


In [5]:
# Exercise information is missing, so lets start by filling it in.
ex1_map = {'A': 'Squat', 'B': 'Squat'}
ex2_map = {'A': 'Bench', 'B': 'Strict Press'}
ex3_map = {'A': 'Bent Over Row', 'B': 'Deadlift'}

df['Exercise 1'] = df['Workout'].map(ex1_map)
df['Exercise 2'] = df['Workout'].map(ex2_map)
df['Exercise 3'] = df['Workout'].map(ex3_map)

In [6]:
def split_exercises(data, n_exercsies=3):
    
    # Start by splitting the exercises
    always_keep_cols = ['Date', 'Workout', 'Body Weight']
    ex_cols = ['Exercise', 'Weight (LB)'] + ['Set {}'.format(i) for i in range(1, 6)] 
    
    ex_data = []
    
    for ex_num in range(1, n_exercsies+1):
        if ex_num == 1:
            curr_cols = ['Exercise {}'.format(ex_num)] + ex_cols[1:]
        else:
            curr_cols = ['Exercise {}'.format(ex_num)] + [c+'.{}'.format(ex_num-1) for c in ex_cols[1:]]

        curr_data = data[always_keep_cols + curr_cols].copy()
        curr_data.columns = always_keep_cols + ex_cols
        curr_data['ex_num'] = ex_num
        ex_data.append(curr_data)

    out_data = pd.concat(ex_data, sort=False).sort_values(['Date', 'ex_num']).reset_index(drop=True)
    return out_data

In [7]:
data = split_exercises(df)

In [8]:
data.head(10)

,Date,Workout,Body Weight,Exercise,Weight (LB),Set 1,Set 2,Set 3,Set 4,Set 5,ex_num
0,2015-02-09,A,175LB,Squat,95,5,5.0,5.0,5.0,5.0,1
1,2015-02-09,A,175LB,Bench,95,5,5.0,5.0,5.0,5.0,2
2,2015-02-09,A,175LB,Bent Over Row,95,5,5.0,5.0,5.0,5.0,3
3,2015-02-11,B,175LB,Squat,100,5,5.0,5.0,5.0,5.0,1
4,2015-02-11,B,175LB,Strict Press,55,5,5.0,5.0,5.0,5.0,2
5,2015-02-11,B,175LB,Deadlift,135,5,NaN,NaN,NaN,NaN,3
6,2015-02-13,A,175LB,Squat,105,5,5.0,5.0,5.0,5.0,1
7,2015-02-13,A,175LB,Bench,100,5,5.0,5.0,5.0,5.0,2
8,2015-02-13,A,175LB,Bent Over Row,100,5,5.0,5.0,5.0,5.0,3
9,2015-02-16,B,175LB,Squat,110,5,5.0,5.0,5.0,5.0,1


In [9]:
data.tail(10)

,Date,Workout,Body Weight,Exercise,Weight (LB),Set 1,Set 2,Set 3,Set 4,Set 5,ex_num
71,2015-04-04,B,170LB,Deadlift,235,4,NaN,NaN,NaN,NaN,3
72,2015-04-07,A,170LB,Squat,205,5,5.0,5.0,5.0,5.0,1
73,2015-04-07,A,170LB,Bench,145,5,5.0,5.0,5.0,5.0,2
74,2015-04-07,A,170LB,Bent Over Row,150,5,5.0,5.0,5.0,5.0,3
75,2015-04-09,B,170LB,Squat,210,5,5.0,5.0,5.0,5.0,1
76,2015-04-09,B,170LB,Strict Press,90,5,5.0,5.0,5.0,5.0,2
77,2015-04-09,B,170LB,Deadlift,235,5,NaN,NaN,NaN,NaN,3
78,2015-04-11,A,170LB,Squat,215,5,5.0,5.0,5.0,4.0,1
79,2015-04-11,A,170LB,Bench,150,4,5.0,3.0,5.0,4.0,2
80,2015-04-11,A,170LB,Bent Over Row,155,5,4.0,5.0,4.0,5.0,3


In [10]:
# I did Deadlifts 5x5 and number of 'reps' == numeber of successful 5-rep sets.
# Unfortunately don't know how many reps done on uncessful sets, so we will leave as NaN

set_cols = ['Set {}'.format(i) for i in range(1, 6)]

def fix_deadlifts(data):

    out = data.copy()
    idx = out.query('Exercise == "Deadlift"').index
    num_successful_sets = out.loc[idx]['Set 1']

    for i in range(6):
        idx1 = idx[num_successful_sets == i]
        out.loc[idx1, set_cols[:i]] = out.loc[idx1, set_cols[:i]].fillna(5)
    
    idx2 = idx[num_successful_sets >= 1]
    
    out.loc[idx2, 'Set 1'] = 5
    
    return out

In [11]:
data = fix_deadlifts(data)

In [12]:
# Need in format of one set per line...
def split_set(data):

    id_vals = ['Date', 'Workout', 'Body Weight', 'Exercise', 'ex_num', 'Weight (LB)'] 
    melt = data.melt(id_vars=id_vals, value_vars=set_cols, var_name='set_num', value_name='reps')
    
    return melt.sort_values(['Date', 'ex_num', 'set_num']).reset_index(drop=True)

In [13]:
first_half = split_set(data)

In [14]:
first_half.tail(30)

,Date,Workout,Body Weight,Exercise,ex_num,Weight (LB),set_num,reps
375,2015-04-09,B,170LB,Squat,1,210,Set 1,5.0
376,2015-04-09,B,170LB,Squat,1,210,Set 2,5.0
377,2015-04-09,B,170LB,Squat,1,210,Set 3,5.0
378,2015-04-09,B,170LB,Squat,1,210,Set 4,5.0
379,2015-04-09,B,170LB,Squat,1,210,Set 5,5.0
380,2015-04-09,B,170LB,Strict Press,2,90,Set 1,5.0
381,2015-04-09,B,170LB,Strict Press,2,90,Set 2,5.0
382,2015-04-09,B,170LB,Strict Press,2,90,Set 3,5.0
383,2015-04-09,B,170LB,Strict Press,2,90,Set 4,5.0
384,2015-04-09,B,170LB,Strict Press,2,90,Set 5,5.0


# Fix 2nd Half of data

In [15]:
# Now lets fix the similar issues that we find in the second half
df1.head(2)

,Date,Note,Workout,Body Weight,Exercise 1,Weight (KG),Weight (LB),Set 1,Set 2,Set 3,...,Set 5.2,28,29,30,31,32,33,34,35,36
0,2015-04-14,NaN,B,168.8LB,NaN,97.5,215,5.0,5.0,5.0,...,NaN,NaN,0.0,0.0,8.0,5.0,3.0,NaN,NaN,NaN
1,2015-04-15,NaN,A,168.8LB,NaN,100.0,220,5.0,5.0,5.0,...,5.0,NaN,0.0,0.0,5.0,5.0,6.0,NaN,NaN,NaN


In [16]:
df1.iloc[:5, 26:]

,Set 4.2,Set 5.2,28,29,30,31,32,33,34,35,36
0,NaN,NaN,NaN,0.0,0.0,8.0,5.0,3.0,NaN,NaN,NaN
1,5.0,5.0,NaN,0.0,0.0,5.0,5.0,6.0,NaN,NaN,NaN
2,NaN,NaN,NaN,0.0,0.0,8.0,5.0,4.0,NaN,NaN,NaN
3,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,0.0,0.0,8.0,4.0,3.0,NaN,NaN,NaN


In [17]:
# So the reason there was a data structure chagnge is that A 4th exercise was added... 
# a quick examination, we can guess which columns correspoind with what and rename them accorindg to the
# established Style, before transforming

ex4_cols = ['Exercise 4', 'Weight (LB).3', 'Weight (KG).3']  + ['Set {}.3'.format(i) for i in range(1, 6)] 
ex4_colname_map = {i: v for i, v in zip(range(28, 36), ex4_cols)}

df1 = df1.rename(columns=ex4_colname_map)
ex4_cols.remove('Weight (KG).3')
df1.head(2)

,Date,Note,Workout,Body Weight,Exercise 1,Weight (KG),Weight (LB),Set 1,Set 2,Set 3,...,Set 5.2,Exercise 4,Weight (LB).3,Weight (KG).3,Set 1.3,Set 2.3,Set 3.3,Set 4.3,Set 5.3,36
0,2015-04-14,NaN,B,168.8LB,NaN,97.5,215,5.0,5.0,5.0,...,NaN,NaN,0.0,0.0,8.0,5.0,3.0,NaN,NaN,NaN
1,2015-04-15,NaN,A,168.8LB,NaN,100.0,220,5.0,5.0,5.0,...,5.0,NaN,0.0,0.0,5.0,5.0,6.0,NaN,NaN,NaN


In [18]:
ex4_map = {'A': 'Dips', 'B': 'Pull Up'}

df1['Exercise 1'] = df1['Workout'].map(ex1_map)
df1['Exercise 2'] = df1['Workout'].map(ex2_map)
df1['Exercise 3'] = df1['Workout'].map(ex3_map)
df1['Exercise 4'] = df1['Workout'].map(ex4_map)

# In 2016, when I restarted, I did power cleans instead of barbell Rows.
new_ex3_map = {'A': 'Power Clean', 'B': 'Deadlift'}
idx = df1.query('Date > {!r}'.format(datetime.date(2016, 1, 1))).index

df1.loc[idx, 'Exercise 3'] = df1.loc[idx, 'Workout'].map(new_ex3_map)

In [19]:
data1 = split_exercises(df1, 4)

In [20]:
data1.head(6)

,Date,Workout,Body Weight,Exercise,Weight (LB),Set 1,Set 2,Set 3,Set 4,Set 5,ex_num
0,2015-04-14,B,168.8LB,Squat,215.0,5.0,5.0,5.0,5.0,5.0,1
1,2015-04-14,B,168.8LB,Strict Press,95.0,5.0,5.0,5.0,5.0,5.0,2
2,2015-04-14,B,168.8LB,Deadlift,245.0,5.0,NaN,NaN,NaN,NaN,3
3,2015-04-14,B,168.8LB,Pull Up,0.0,8.0,5.0,3.0,NaN,NaN,4
4,2015-04-15,A,168.8LB,Squat,220.0,5.0,5.0,5.0,5.0,5.0,1
5,2015-04-15,A,168.8LB,Bench,150.0,5.0,1.0,5.0,5.0,5.0,2


In [21]:
data1 = fix_deadlifts(data1)
data1.query('Exercise == "Deadlift"')

,Date,Workout,Body Weight,Exercise,Weight (LB),Set 1,Set 2,Set 3,Set 4,Set 5,ex_num
2,2015-04-14,B,168.8LB,Deadlift,245.0,5.0,5.0,5.0,5.0,5.0,3
10,2015-04-18,B,168.8LB,Deadlift,255.0,5.0,5.0,5.0,NaN,NaN,3
18,2015-04-28,B,172.8LB,Deadlift,255.0,5.0,5.0,5.0,5.0,5.0,3
26,2015-05-02,B,172.8LB,Deadlift,265.0,5.0,5.0,5.0,5.0,5.0,3
34,2015-05-07,B,173.6LB,Deadlift,270.0,5.0,5.0,5.0,5.0,5.0,3
42,2015-05-14,B,173.6LB,Deadlift,275.0,5.0,5.0,5.0,5.0,5.0,3
50,2015-05-21,B,174LB,Deadlift,280.0,5.0,5.0,NaN,NaN,NaN,3
58,2015-05-26,B,174LB,Deadlift,280.0,5.0,5.0,5.0,5.0,NaN,3
66,2015-05-30,B,175LB,Deadlift,280.0,5.0,5.0,5.0,5.0,5.0,3
74,2015-06-05,B,175LB,Deadlift,285.0,5.0,5.0,5.0,5.0,5.0,3


In [22]:
# Power cleans were performed 5x3... so 1-2 == 1 rep, 3-4 == 2 reps, and 5 == 3 reps.

def fix_power_clean(data):
    rep_map = {0: 0, 1: 1, 2: 1, 3: 2, 4: 2, 5: 3}
    
    out = data.copy()
    pc_idx = out.query('Exercise == "Power Clean"').index
    
    for i in range(6):
        out.loc[pc_idx, set_cols] = out.loc[pc_idx, set_cols].replace(i, rep_map[i])
        
    return out

In [23]:
data1 = fix_power_clean(data1)

In [24]:
second_half = split_set(data1)
second_half.head(20)

,Date,Workout,Body Weight,Exercise,ex_num,Weight (LB),set_num,reps
0,2015-04-14,B,168.8LB,Squat,1,215.0,Set 1,5.0
1,2015-04-14,B,168.8LB,Squat,1,215.0,Set 2,5.0
2,2015-04-14,B,168.8LB,Squat,1,215.0,Set 3,5.0
3,2015-04-14,B,168.8LB,Squat,1,215.0,Set 4,5.0
4,2015-04-14,B,168.8LB,Squat,1,215.0,Set 5,5.0
5,2015-04-14,B,168.8LB,Strict Press,2,95.0,Set 1,5.0
6,2015-04-14,B,168.8LB,Strict Press,2,95.0,Set 2,5.0
7,2015-04-14,B,168.8LB,Strict Press,2,95.0,Set 3,5.0
8,2015-04-14,B,168.8LB,Strict Press,2,95.0,Set 4,5.0
9,2015-04-14,B,168.8LB,Strict Press,2,95.0,Set 5,5.0


# Reformat into similar to Strong...

In [25]:
all_data = pd.concat([first_half, second_half], sort=False)

In [26]:
all_data = all_data.dropna(subset=['reps'])
all_data['Workout'] = all_data['Workout'].apply(lambda s: 'Sronglifts ' + s)
all_data['reps'] = all_data['reps'].astype(int)

In [27]:
all_data['set_num'] = all_data['set_num'].apply(lambda s: int(s.split(' ')[-1])-1)

In [28]:
strong = pd.read_csv('data/workout-data/02-04-18-strong.csv')
strong.head(2)

,Date,Workout Name,Exercise Name,Set Order,kg,Reps,mi,Seconds,Notes
0,2015-05-21 08:47:29,Free Workout,Squat,0,99.79,5,0.0,0,NaN
1,2015-05-21 08:47:29,Free Workout,Squat,1,99.79,5,0.0,0,NaN


In [29]:
# we only care about the column names, but we're using lb, so if kg in this data, change
strong = strong.rename(columns={'kg': 'lb'})
strong_cols = strong.columns

In [30]:
# Map to old Strong export version
name_map = {
    'Workout': 'Workout Name',
    'Exercise': 'Exercise Name',
    'set_num': 'Set Order',
    'Weight (LB)': 'lb',
    'reps': 'Reps'}

In [31]:
all_data = all_data.rename(columns=name_map)
for c in strong_cols:
    if c not in all_data:
        all_data[c] = float('nan')

all_data = all_data[strong_cols]
all_data

,Date,Workout Name,Exercise Name,Set Order,lb,Reps,mi,Seconds,Notes
0,2015-02-09,Sronglifts A,Squat,0,95.0,5,NaN,NaN,NaN
1,2015-02-09,Sronglifts A,Squat,1,95.0,5,NaN,NaN,NaN
2,2015-02-09,Sronglifts A,Squat,2,95.0,5,NaN,NaN,NaN
3,2015-02-09,Sronglifts A,Squat,3,95.0,5,NaN,NaN,NaN
4,2015-02-09,Sronglifts A,Squat,4,95.0,5,NaN,NaN,NaN
5,2015-02-09,Sronglifts A,Bench,0,95.0,5,NaN,NaN,NaN
6,2015-02-09,Sronglifts A,Bench,1,95.0,5,NaN,NaN,NaN
7,2015-02-09,Sronglifts A,Bench,2,95.0,5,NaN,NaN,NaN
8,2015-02-09,Sronglifts A,Bench,3,95.0,5,NaN,NaN,NaN
9,2015-02-09,Sronglifts A,Bench,4,95.0,5,NaN,NaN,NaN


In [32]:
all_data.to_csv('data/workout-data/stronglifts_formatted_as_strong.csv', index=False)